In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [36]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
# pip install -U mittens

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from mittens import GloVe, Mittens

In [39]:
# pip install autocorrect

In [40]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [41]:
# nltk.download('all')

In [42]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [43]:
# nltk.download('all')


## Loading Data

In [44]:
# data_path = r'/content/drive/My Drive/Research Project/Data'

In [45]:
data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [46]:
obama_train_pr_df = pd.read_csv(os.path.join(data_path, 'Obama Training1 Data.csv'), usecols = [1,2])
# romney_train_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training1 Data.csv'), usecols = [1,2])

In [47]:
obama_train_pr_df.head()

,Doc Text,Sentiment
0,kirkpatrick wear baseball cap embroider obama ...,Neutral
1,obama debate cracker as cracker tonight tune t...,Positive
2,miss point afraid understand big picture dont ...,Neutral
3,raise democrat leave party year ago lifetime n...,Negative
4,obama camp can not afford low expectation toni...,Neutral


In [48]:
obama_train_pr_df = obama_train_pr_df.dropna()
# romney_train_pr_df = romney_train_pr_df.dropna()

### Converting into TF-IDF

In [49]:
def tf_idf(x):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(x)
    #print(tfidf_matrix)
    x1 = tfidf_matrix.toarray()
#     print(x1)
    return x1

In [50]:
obama_train_pr_tfidf = tf_idf(obama_train_pr_df['Doc Text'])
# romney_train_pr_tfidf = tf_idf(romney_train_pr_df['Doc Text'])

In [51]:
obama_train_pr_tfidf.shape

(5470, 6938)

### Cross Valid Function

In [52]:
def cross_valid(X,y,model):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        model = model.fit(X_train1, temp)

        y_pred = model.predict(X_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Cross Valid func with PCA

In [53]:
def cross_valid_PCA(X,y,model, n):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    
    pca = PCA(n_components=n)
    
    
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        x_pca1 = pca.fit_transform(X_train1)
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        model = model.fit(x_pca1, temp)
        
        x_pca_val = pca.transform(X_val)
        y_pred = model.predict(x_pca_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:


# # clf = SVC(kernel="rbf", gamma=1, class_weight='balanced')
# #     error = 1-(cross_val_score(clf, x_pca1, temp, cv = 5, scoring = 'f1_macro'))
# #     f1 = mean(error) + stdev(error) 
# #print(f1)


# clf = LogisticRegression(class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# # print([f1_list[2],f1_list[1],f1_list[0]])

# print(eval_dict)


In [56]:
# With Bayesian Optimization, and LR

def objective_func_LR(args):

    C = args['C']
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']

 
    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    return -(eval_score)


space = {'C': hp.uniform('C', 0,10),
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR)

  0%|          | 0/10 [01:03<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [35]:
# With PCA, Bayesian Optimization, and LR

def objective_func_LR_PCA(args):

    C = args['C']
    n = args['n']
    
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']


    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    par_dict = {'C': C, 'n': n}
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(par_dict)
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
         'n': hp.choice('n', np.arange(100, 500, step =50))
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),
        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR_PCA = fmin(objective_func_LR_PCA, space, algo=tpe.suggest, max_evals=20)


{'C': 0.882777178013636, 'n': 200}                    
{'accuracy': 0.561, 'f1_pos': 0.572, 'f1_neu': 0.517, 'f1_neg': 0.592, 'eval_score': 0.575}
{'C': 1.2318460206237267, 'n': 400}                                 
{'accuracy': 0.578, 'f1_pos': 0.597, 'f1_neu': 0.531, 'f1_neg': 0.606, 'eval_score': 0.594}
{'C': 0.032559308210585725, 'n': 250}                               
{'accuracy': 0.556, 'f1_pos': 0.562, 'f1_neu': 0.498, 'f1_neg': 0.599, 'eval_score': 0.572}
{'C': 1.584830077996315, 'n': 100}                                  
{'accuracy': 0.547, 'f1_pos': 0.547, 'f1_neu': 0.513, 'f1_neg': 0.578, 'eval_score': 0.557}
{'C': 0.4926397918121812, 'n': 150}                                 
{'accuracy': 0.559, 'f1_pos': 0.557, 'f1_neu': 0.526, 'f1_neg': 0.593, 'eval_score': 0.57}
{'C': 0.5762716880453709, 'n': 450}                                 
{'accuracy': 0.576, 'f1_pos': 0.586, 'f1_neu': 0.533, 'f1_neg': 0.61, 'eval_score': 0.591}
{'C': 1.4182001918278502, 'n': 300}               

In [32]:
# # With Bayesian Optimization, different class weights and LR

# def objective_func_LR(args):

#     C = args['C']
#     balance = args['balance']
#     # multi_class = args['multi_class']
#     # l1_ratio = args['l1_ratio']

 
#     clf = LogisticRegression(C= C,class_weight=balance,n_jobs =-1)
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,10),
#          'balance': hp.choice('balance', [{1:3, 2:1, 3:2}, {1:15, 2:1, 3:10}, {1:20, 2:5, 3:10}, {1:100, 2:10, 3:100}])
#         # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
#         # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

#         #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
#         }                        
                                
                                
# best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_LR)

### SVM

In [50]:
# C = 1
# n = 3
# kernel = 'rbf'

# par_dict = {'C': C, 'n': n, 'kernel': kernel}
# print(par_dict)

# clf = SVC(C= C, kernel = kernel, class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print(eval_dict)
# print('\n')    
# return -(eval_score)

In [51]:
# C = 2
# n = 3
# kernel = 'rbf'

# par_dict = {'C': C, 'n': n, 'kernel': kernel}
# print(par_dict)

# clf = SVC(C= C, kernel = kernel, class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print(eval_dict)
# print('\n')    
# return -(eval_score)

In [52]:
# C = 3
# n = 3
# kernel = 'rbf'

# par_dict = {'C': C, 'n': n, 'kernel': kernel}
# print(par_dict)

# clf = SVC(C= C, kernel = kernel, class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print(eval_dict)
# print('\n')    
# return -(eval_score)

In [47]:
# With PCA, Bayesian Optimization, and SVM

def objective_func_SVM_PCA(args):
    C = args['C']
    n = args['n']
#     kernel = args['kernel']
#     gamma = args['gamma']
#     degree = args['degree']

    par_dict = {'C': C, 'n': n, 'kernel': 'rbf'}
    print(par_dict)
 
    clf = SVC(C= C, kernel = 'rbf', class_weight='balanced')
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,1),
         'n': hp.choice('n', np.arange(3, 1003, step =100))
#         'kernel': hp.choice('kernel', ['poly', 'rbf']) 
#         'gamma': hp.choice('gamma',range(1,4)),
#          'degree' : hp.choice('degree',range(1,4))
        }
                                
                                
                                
best_classifier_SVM_PCA = fmin(objective_func_SVM_PCA, space, algo=tpe.suggest, max_evals=20)
print(best_classifier_SVM_PCA)

{'C': 0.2892192714238556, 'n': 603, 'kernel': 'rbf'}  
{'accuracy': 0.576, 'f1_pos': 0.565, 'f1_neu': 0.561, 'f1_neg': 0.6, 'eval_score': 0.58}
{'C': 0.5151888680639615, 'n': 203, 'kernel': 'rbf'}                
{'accuracy': 0.57, 'f1_pos': 0.578, 'f1_neu': 0.547, 'f1_neg': 0.588, 'eval_score': 0.579}
{'C': 0.8193955283313585, 'n': 503, 'kernel': 'rbf'}                
{'accuracy': 0.588, 'f1_pos': 0.591, 'f1_neu': 0.565, 'f1_neg': 0.611, 'eval_score': 0.597}
{'C': 0.8798157958089573, 'n': 303, 'kernel': 'rbf'}                 
{'accuracy': 0.579, 'f1_pos': 0.588, 'f1_neu': 0.557, 'f1_neg': 0.592, 'eval_score': 0.586}
{'C': 0.8809658735583068, 'n': 603, 'kernel': 'rbf'}                 
{'accuracy': 0.588, 'f1_pos': 0.597, 'f1_neu': 0.567, 'f1_neg': 0.602, 'eval_score': 0.596}
{'C': 0.7960948785793291, 'n': 803, 'kernel': 'rbf'}                 
{'accuracy': 0.593, 'f1_pos': 0.599, 'f1_neu': 0.567, 'f1_neg': 0.615, 'eval_score': 0.602}
{'C': 0.4684465835596907, 'n': 203, 'kernel': 'rb

In [ ]:
{'C': 2.645341878923651, 'n': 3, 'kernel': 'rbf'}                       
{'accuracy': 0.597, 'f1_pos': 0.607, 'f1_neu': 0.553, 'f1_neg': 0.631, 'eval_score': 0.612}

### KNN

In [52]:
# WIth Bayesian Optimization and KNN
def objective_func_KNN(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']  

    par_dict = {'n_neighbors': n_neighbors, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,31, step =3)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])
        }

best_classifier_KNN = fmin(objective_func_KNN, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_KNN)

{'n_neighbors': 19, 'metric': 'manhattan'}            
{'accuracy': 0.366, 'f1_pos': 0.488, 'f1_neu': 0.329, 'f1_neg': 0.041, 'eval_score': 0.298}
{'n_neighbors': 13, 'metric': 'manhattan'}                           
{'accuracy': 0.371, 'f1_pos': 0.49, 'f1_neu': 0.298, 'f1_neg': 0.107, 'eval_score': 0.323}
{'n_neighbors': 16, 'metric': 'euclidean'}                           
{'accuracy': 0.505, 'f1_pos': 0.561, 'f1_neu': 0.426, 'f1_neg': 0.5, 'eval_score': 0.522}
{'n_neighbors': 16, 'metric': 'manhattan'}                           
{'accuracy': 0.362, 'f1_pos': 0.485, 'f1_neu': 0.306, 'f1_neg': 0.064, 'eval_score': 0.304}
{'n_neighbors': 13, 'metric': 'euclidean'}                           
{'accuracy': 0.503, 'f1_pos': 0.561, 'f1_neu': 0.424, 'f1_neg': 0.497, 'eval_score': 0.52}
{'n_neighbors': 1, 'metric': 'euclidean'}                            
{'accuracy': 0.486, 'f1_pos': 0.517, 'f1_neu': 0.446, 'f1_neg': 0.492, 'eval_score': 0.498}
{'n_neighbors': 1, 'metric': 'euclidean'}      

In [53]:
# WIth PCA , Bayesian Optimization and KNN
def objective_func_KNN_PCA(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    n = args['n']    

    par_dict = {'n_neighbors': n_neighbors, 'n': n, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,10, step =1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"]),
        'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_KNN_PCA = fmin(objective_func_KNN_PCA, space, algo=tpe.suggest, max_evals=20)


{'n_neighbors': 3, 'n': 33, 'metric': 'euclidean'}    
{'accuracy': 0.462, 'f1_pos': 0.474, 'f1_neu': 0.458, 'f1_neg': 0.451, 'eval_score': 0.462}
{'n_neighbors': 6, 'n': 3, 'metric': 'euclidean'}                   
{'accuracy': 0.439, 'f1_pos': 0.477, 'f1_neu': 0.406, 'f1_neg': 0.435, 'eval_score': 0.45}
{'n_neighbors': 3, 'n': 13, 'metric': 'manhattan'}                  
{'accuracy': 0.451, 'f1_pos': 0.479, 'f1_neu': 0.431, 'f1_neg': 0.442, 'eval_score': 0.457}
{'n_neighbors': 1, 'n': 53, 'metric': 'euclidean'}                  
{'accuracy': 0.462, 'f1_pos': 0.463, 'f1_neu': 0.456, 'f1_neg': 0.467, 'eval_score': 0.464}
{'n_neighbors': 8, 'n': 28, 'metric': 'euclidean'}                  
{'accuracy': 0.488, 'f1_pos': 0.506, 'f1_neu': 0.492, 'f1_neg': 0.464, 'eval_score': 0.486}
{'n_neighbors': 5, 'n': 8, 'metric': 'euclidean'}                   
{'accuracy': 0.452, 'f1_pos': 0.48, 'f1_neu': 0.437, 'f1_neg': 0.438, 'eval_score': 0.457}
{'n_neighbors': 1, 'n': 38, 'metric': 'manhattan'}

### Naive Bayes

In [53]:
clf = GaussianNB()
accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
print(eval_dict)

{'accuracy': 0.467, 'f1_pos': 0.527, 'f1_neu': 0.34, 'f1_neg': 0.497, 'eval_score': 0.497}


In [55]:
# With PCA and Naive bayes
def objective_func_NB_PCA(args):
    n = args['n']

    par_dict = {'n': n}
    print(par_dict)
    
    clf = GaussianNB()
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_NB_PCA = fmin(objective_func_NB_PCA, space, algo=tpe.suggest, max_evals=10)



{'n': 18}                                             
{'accuracy': 0.459, 'f1_pos': 0.426, 'f1_neu': 0.317, 'f1_neg': 0.556, 'eval_score': 0.48}
{'n': 28}                                                          
{'accuracy': 0.472, 'f1_pos': 0.427, 'f1_neu': 0.374, 'f1_neg': 0.56, 'eval_score': 0.486}
{'n': 28}                                                           
{'accuracy': 0.463, 'f1_pos': 0.431, 'f1_neu': 0.359, 'f1_neg': 0.548, 'eval_score': 0.481}
{'n': 18}                                                           
{'accuracy': 0.458, 'f1_pos': 0.426, 'f1_neu': 0.308, 'f1_neg': 0.557, 'eval_score': 0.48}
{'n': 3}                                                            
{'accuracy': 0.407, 'f1_pos': 0.341, 'f1_neu': 0.143, 'f1_neg': 0.541, 'eval_score': 0.43}
{'n': 28}                                                           
{'accuracy': 0.465, 'f1_pos': 0.433, 'f1_neu': 0.38, 'f1_neg': 0.534, 'eval_score': 0.477}
{'n': 28}                                             

## Ensemble Methods

### Random Forest

In [56]:
# With Bayesian Optimization, and Random Forest

def objective_func_RF(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_RF = fmin(objective_func_RF, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_RF)

{'n_estimators': 61, 'max_depth': 43, 'min_samples_split': 3, 'min_samples_leaf': 2}
{'accuracy': 0.562, 'f1_pos': 0.56, 'f1_neu': 0.529, 'f1_neg': 0.595, 'eval_score': 0.572}
{'n_estimators': 74, 'max_depth': 84, 'min_samples_split': 3, 'min_samples_leaf': 8}
{'accuracy': 0.551, 'f1_pos': 0.543, 'f1_neu': 0.518, 'f1_neg': 0.588, 'eval_score': 0.561}
{'n_estimators': 74, 'max_depth': 96, 'min_samples_split': 8, 'min_samples_leaf': 4}
{'accuracy': 0.563, 'f1_pos': 0.56, 'f1_neu': 0.526, 'f1_neg': 0.599, 'eval_score': 0.574}
{'n_estimators': 98, 'max_depth': 71, 'min_samples_split': 11, 'min_samples_leaf': 2}
{'accuracy': 0.566, 'f1_pos': 0.558, 'f1_neu': 0.534, 'f1_neg': 0.603, 'eval_score': 0.576}
{'n_estimators': 20, 'max_depth': 74, 'min_samples_split': 4, 'min_samples_leaf': 7}
{'accuracy': 0.55, 'f1_pos': 0.551, 'f1_neu': 0.516, 'f1_neg': 0.583, 'eval_score': 0.561}
{'n_estimators': 80, 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 4}
{'accuracy': 0.561, 'f1_pos': 0.

In [58]:
# With PCA,  Bayesian Optimization, and Random Forest

def objective_func_RF_PCA(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

{'n': 183, 'n_estimators': 87, 'max_depth': 61, 'min_samples_split': 8, 'min_samples_leaf': 4}
{'accuracy': 0.552, 'f1_pos': 0.542, 'f1_neu': 0.526, 'f1_neg': 0.585, 'eval_score': 0.56}
{'n': 163, 'n_estimators': 21, 'max_depth': 32, 'min_samples_split': 7, 'min_samples_leaf': 2}
{'accuracy': 0.517, 'f1_pos': 0.515, 'f1_neu': 0.498, 'f1_neg': 0.537, 'eval_score': 0.523}
{'n': 43, 'n_estimators': 93, 'max_depth': 51, 'min_samples_split': 5, 'min_samples_leaf': 2}
{'accuracy': 0.546, 'f1_pos': 0.56, 'f1_neu': 0.514, 'f1_neg': 0.565, 'eval_score': 0.557}
{'n': 43, 'n_estimators': 76, 'max_depth': 53, 'min_samples_split': 7, 'min_samples_leaf': 11}
{'accuracy': 0.535, 'f1_pos': 0.539, 'f1_neu': 0.497, 'f1_neg': 0.569, 'eval_score': 0.548}
{'n': 183, 'n_estimators': 28, 'max_depth': 74, 'min_samples_split': 9, 'min_samples_leaf': 3}
{'accuracy': 0.527, 'f1_pos': 0.533, 'f1_neu': 0.495, 'f1_neg': 0.552, 'eval_score': 0.537}
{'n': 103, 'n_estimators': 22, 'max_depth': 87, 'min_samples_split':

In [ ]:
# # WITH LDA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from math import sqrt
# lda = LinearDiscriminantAnalysis(n_components = None)
# x_lda = lda.fit_transform(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
#     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda, temp, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# obama_train2_pr_tfidf, obama_val_pr_tfidf, obama_train2_pr_class, obama_val_pr_class = train_test_split(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'], test_size = 0.2)

In [ ]:
# x_lda2 = lda.fit_transform(obama_train2_pr_tfidf, obama_train2_pr_class)

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
# #     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda2, obama_train2_pr_class, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# x_val_lda = lda.transform(obama_val_pr_tfidf)
# x_val_lda_df = pd.DataFrame(data = x_val_lda)

In [ ]:
# c = lambda b : 1 if b == 0 else b
# d = lambda b : 2 if b <= 1 else b
# bc = RandomForestClassifier(max_depth = c(best_classifier['max_depth']), min_samples_leaf = c(best_classifier['min_samples_leaf']), min_samples_split = d(best_classifier['min_samples_split'])
#                             ,n_estimators = best_classifier['n_estimators'])

# bc.fit(x_lda2, obama_train2_pr_class)

# y_pred = bc.predict(x_val_lda)

# accuracy_score(obama_val_pr_class, y_pred)

In [ ]:
# lda_clf = lda.fit(obama_train2_pr_tfidf, obama_train2_pr_class)
# y_pred_lda_clf = lda_clf.predict(obama_val_pr_tfidf)
# # y_pred_lda_clf

# accuracy_score(obama_val_pr_class, y_pred_lda_clf)

### Light Gradient Boosting

In [19]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier

In [28]:
# With Bayesian Optimization, and Light GB

def objective_func_LGB(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
    subsample = args['subsample']
    min_split_gain = args['min_split_gain']

    par_dict = {'boosting_type': boosting_type, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': subsample, 'min_split_gain': min_split_gain}
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=subsample, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
     'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1))}
                                
                                
                                
best_classifier_LGB = fmin(objective_func_LGB, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_LGB)

{'boosting_type': 'goss', 'n_estimators': 170, 'max_depth': 7, 'learning_rate': 0.06041058628039278, 'subsample': 0.8666666666666667, 'min_split_gain': 4}
{'accuracy': 0.525, 'f1_pos': 0.551, 'f1_neu': 0.481, 'f1_neg': 0.542, 'eval_score': 0.539}
{'boosting_type': 'gbdt', 'n_estimators': 219, 'max_depth': 8, 'learning_rate': 0.115917144381938, 'subsample': 1.0, 'min_split_gain': 4}
{'accuracy': 0.531, 'f1_pos': 0.557, 'f1_neu': 0.466, 'f1_neg': 0.565, 'eval_score': 0.551}
{'boosting_type': 'gbdt', 'n_estimators': 281, 'max_depth': 8, 'learning_rate': 0.10018895580687695, 'subsample': 0.9333333333333333, 'min_split_gain': 4}
{'accuracy': 0.529, 'f1_pos': 0.561, 'f1_neu': 0.453, 'f1_neg': 0.566, 'eval_score': 0.552}
{'boosting_type': 'goss', 'n_estimators': 441, 'max_depth': 5, 'learning_rate': 0.11625015294913124, 'subsample': 1.0, 'min_split_gain': 0}
{'accuracy': 0.516, 'f1_pos': 0.538, 'f1_neu': 0.477, 'f1_neg': 0.533, 'eval_score': 0.529}
{'boosting_type': 'dart', 'n_estimators': 28

In [30]:
# With PCA, Bayesian Optimization, and Light GB

def objective_func_LGB_PCA(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
#     subsample = args['subsample']
    min_split_gain = args['min_split_gain']
    n = args['n']
    
    par_dict = {'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': 1, 'min_split_gain': min_split_gain} 
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=1, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
#      'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))}
                                
                                
                                
best_classifier_LGB_PCA = fmin(objective_func_LGB_PCA, space, algo=tpe.suggest, max_evals=35)
print(best_classifier_LGB_PCA)

{'n': 103, 'n_estimators': 63, 'max_depth': 8, 'learning_rate': 0.16383257483998023, 'subsample': 1, 'min_split_gain': 0}
{'accuracy': 0.524, 'f1_pos': 0.537, 'f1_neu': 0.495, 'f1_neg': 0.541, 'eval_score': 0.534}
{'n': 143, 'n_estimators': 238, 'max_depth': 8, 'learning_rate': 0.007000130544041383, 'subsample': 1, 'min_split_gain': 5}
{'accuracy': 0.533, 'f1_pos': 0.552, 'f1_neu': 0.497, 'f1_neg': 0.551, 'eval_score': 0.545}
{'n': 143, 'n_estimators': 310, 'max_depth': 7, 'learning_rate': 0.08240929053331739, 'subsample': 1, 'min_split_gain': 3}
{'accuracy': 0.555, 'f1_pos': 0.57, 'f1_neu': 0.523, 'f1_neg': 0.574, 'eval_score': 0.566}
{'n': 123, 'n_estimators': 281, 'max_depth': 7, 'learning_rate': 0.1944260906828903, 'subsample': 1, 'min_split_gain': 0}
{'accuracy': 0.534, 'f1_pos': 0.539, 'f1_neu': 0.507, 'f1_neg': 0.559, 'eval_score': 0.544}
{'n': 23, 'n_estimators': 220, 'max_depth': 6, 'learning_rate': 0.17933083294492252, 'subsample': 1, 'min_split_gain': 4}
{'accuracy': 0.5, 'f

### Gradient Boosting

In [31]:
# With Bayesian Optimization, and GB

def objective_func_GB(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = { 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = GradientBoostingClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_GB= fmin(objective_func_GB, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_GB)

{'n_estimators': 76, 'max_depth': 91, 'min_samples_split': 6, 'min_samples_leaf': 4}
  0%|          | 0/30 [33:53<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [ ]:
# With PCA, Bayesian Optimization, and GB

def objective_func_GB(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(sampling_strategy='not majority',random_state=42)
romney_train_pr_tfidf_smote, romney_train_pr_class = sm.fit_sample(romney_train_pr_tfidf, romney_train_pr_df['Sentiment'] )


sm = SMOTE(sampling_strategy='not majority',random_state=42)
obama_train_pr_tfidf_smote, obama_train_pr_class = sm.fit_sample(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])